David Mykytyn and Alex Malz

Goals and scope: 

In [ ]:
from itertools import product, combinations, count, permutations, starmap, chain, repeat
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import sklearn
from sklearn.neighbors import KernelDensity as KD
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
import corner
from sklearn.mixture import GaussianMixture as GM
from sklearn.mixture import BayesianGaussianMixture as BGM
from sklearn.pipeline import Pipeline
from scipy.stats import norm
from matplotlib.colors import LogNorm
from astroquery.gaia import Gaia
from scipy import stats
import seaborn as sns

In [ ]:
import Herd

In [ ]:
%matplotlib inline

Good `'Cluster'`s to choose from: Alessi\_24, ASCC_99, Alessi\_12

In [ ]:
cluster=Herd.get_cluster()

In [ ]:
the_color = 'BP-RP'
the_mag = 'phot_rp_mean_mag'

In [ ]:
plt.hist(cluster['PMemb'])
plt.xlabel('PMemb')
plt.title('cluster membership probability of '+cluster['Cluster'][0])

In [ ]:
plt.plot(cluster[the_color],cluster[the_mag], '+')
plt.ylim(19, 7)
plt.xlabel('color')
plt.ylabel('magnitude')
plt.title('color-magnitude diagram of '+cluster['Cluster'][0])

PCA - diagram, inverse of variance as metric -> sqrt(thing)= transformation, apply ->PCA(test) -> KDE -> inverse transform both L and R on Kernel Widths

LLE? -> Local linear embedding -> for non MS

remove outliers before PCA

In [ ]:
xmin, xmax = (-4., 4.)
ymin, ymax = (-4., 4.)
xlen, ylen = (50, 50)

In [ ]:
xlims = xmin, xmax
ylims = ymin, ymax
ranges = xlen, ylen

In [ ]:
pipe = Herd.pca_kde_pipe()
pipe = Herd.pca_kde_fit(pipe, cluster[the_color], cluster[the_mag])
xlin, ylin, xgrid, ygrid = Herd.gen_grid(xlims, ylims, ranges)
lins = xlin, ylin
grids = xgrid, ygrid
logdens = Herd.pca_kde_score(pipe, lins)

In [ ]:
plt.contour(xgrid, ygrid, np.exp(logdens));

In [ ]:
α, β = Herd.kde_ipca(pipe, grids, ranges)

In [ ]:
plt.plot(cluster[the_color], cluster[the_mag], 'r+',alpha=.4)
plt.contour(α, β, np.exp(logdens))
plt.ylim(19,8)
plt.xlabel(the_color)
plt.ylabel(the_mag)
plt.title('color-magnitude diagram of '+cluster['Cluster'][0])

In [ ]:
newmags, newcolors = Herd.get_added_samps(pipe, 100)

### do the newly-defined transforms here and plot that instead of KDE in color-mag space

In [ ]:
doublepipe = Herd.pca_kde_fit(Herd.pca_kde_pipe(), newcolors[::10], newmags[::10])
double_logdens = Herd.pca_kde_score(doublepipe, lins)

In [ ]:
α2, β2 = Herd.kde_ipca(doublepipe, grids, ranges)

In [ ]:
plt.plot(cluster[the_color], cluster[the_mag], 'r+',alpha=.4)
plt.contour(α2, β2, np.exp(double_logdens))
plt.ylim(19,8)
plt.xlabel(the_color)
plt.ylabel(the_mag)
plt.title('color-magnitude diagram of '+cluster['Cluster'][0])

In [ ]:
s_KDE = pipe[1]
d_KDE = doublepipe[1]

In [ ]:
X = Herd.samps_vstack(cluster[the_color],cluster[the_mag])
X_transform = pipe[0].transform(X)

In [ ]:
def get_scores(apipe, colors, mags):
    X = Herd.samps_vstack(colors,mags)
    X_transform = apipe[0].transform(X)
    return apipe[-1].score_samples(X_transform)

In [ ]:
def wget_scores(apipe):
    return get_scores(apipe, cluster[the_color], cluster[the_mag])

In [ ]:
sscores = wget_scores(pipe)
dscores = wget_scores(doublepipe)

In [ ]:
colors = np.logaddexp(-sscores,dscores)


In [ ]:
plt.scatter(cluster[the_color], cluster[the_mag], c=colors, marker='+',)
plt.ylim(19,8)


### END OF REAL WORK

Write a title and abstract  
NOTES FROM MEETING (05/21):  
First, talk to Boris  
Gaia has metadata for velocity variations (= uncertainty)  
    - Label every star with ratio of KDEs (at loc of star) 
    - difference in log prob
    - (dumb) cut in prob ratio 
    - get a real single star KDE
    - do everything again and see if changes

Stars that are low probability under both KDEs -> Triples?  
Plots 
    1. colored under delta log-prob
    2. Max of the two log probs
LogAddExp (SoftMax)

When do people use CMD?
 - Infer age, metallicity (not for us)
 - Search for structure (disrupted clusters, etc)
 - Binary fractions (double star CMD?)
 - Refine cluster membership
 
Email Jackie at AMNH

Simple use-case: "Our method: We demonstrate this with etc":  
See:  
    - Anderson, et. al (mixture of Gaussian)
    - Boris, Hogg
    - Widmark